In [9]:
import torch
import math

import matplotlib.pyplot as plt
# import openturns as ot
import numpy as np

import torch.nn as nn

import random

import torch.nn.functional as F

In [11]:
# X = [14, 12, 13, 12]
# outDim = [10,10]

# test = SocialSig(X)

In [64]:
class SocialSig(torch.nn.Module):
    '''
    Class to create the social signature image
    '''
    def __init__(self, coords):
        '''
        Basic steps for class:
            1. Randomly initalize 'weights' which I believe are actually the coords of the points
            2. Train kriggin model to interpolate points in between
            3. Predict what the points would be within a 224x224 matrix and output the resulting matrix
        '''
        super(SocialSig, self).__init__()
        self.outDim = [10,10]
#         self.coords = torch.nn.Parameter(torch.tensor(random.choices(range(0,outDim[1]), k=16), dtype=torch.float32)) 
        # self.register_parameter(name = 'coords', param = self.coords)

    def forward(self, input, coords):
        self.X = input
        self.grid = self.__make_blank_coord_grid()
        tensorRet = self.IDW(coords)
        return tensorRet
       
    
    def IDW(self, coords):
        '''
        Train the IDW model to predict all of the points that are between known points
        '''
        coords = torch.clamp(coords, min=0, max=self.outDim[1])
        
        for cell in range(0, len(self.grid)):
            weightedVals = []
            for column in range(0, len(X)):
                xCoordLookup = column * 2
                yCoordLookup = xCoordLookup + 1
            

                measurementCellValue = self.X[column]

                estCellX = self.grid[cell][0]
                estCellY = self.grid[cell][1]

                measureCellX = coords[xCoordLookup]
                measureCellY = coords[yCoordLookup]

                A2 = abs(estCellX - measureCellX)**2
                B2 = abs(estCellY - measureCellY)**2
                C2 = math.sqrt(A2+B2) 
                if(C2 == 0):
                    C2 = 1
                
                weightedVals.append(measurementCellValue[0] * (1/(C2**2)))
            self.grid[cell] = sum(weightedVals)
        numpyGrid = torch.from_numpy(np.reshape(np.array(self.grid), (1,1,self.outDim[0],self.outDim[1])))
        tensorGrid = torch.tensor(numpyGrid, dtype=torch.float)
        # print(self.coords)

        return tensorGrid 
        
    def __make_blank_coord_grid(self):
        '''
        Make a blank coordinate grid to fill in with real data later
        '''
        return [[x,y] for x in range(0, outDim[0]) for y in range(0,outDim[1])]

In [88]:
# THIS IS WHAT YOU WILL USE TO CREATE YOUR INPUT LAYER--THIS SHOULD BE A LIST WITH 81 ELEMENTS (THE ROW IN THE DF)
def make_example_data(seed, size):
    '''
    Create the training dataset by randomly assigning coordiantes to the known data (i.e the data within a row)
    '''
    points = [[i] for i in list(np.random.randint(low = 0, high = 255, size = size).flatten())]
    return points


make_example_data(14, size=4)

[[226], [102], [41], [193]]

In [89]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
#         self.coords = torch.nn.Parameter(torch.tensor(random.choices(range(0,outDim[1]), k=16), dtype=torch.float32)) 
        self.coords = torch.nn.Parameter(torch.empty(outDim[1]))
        nn.init.uniform_(self.coords, a = 0, b = 8)   

        self.SocialSig = SocialSig(self.coords)
        self.conv1 = nn.Conv2d(1, 10, 10, 1)
        self.linear1 = torch.nn.Linear(10, 1)
        self.linear2 = torch.nn.Linear(2, 1)
        self.alpha = nn.Parameter(torch.tensor(0.5, requires_grad=True))


    def forward(self, input):
        # print(input)
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        print(self.coords)
        sig = self.SocialSig(input, self.coords)#.forward(input, self.coords)
#         print(sig)
        sig = self.conv1(sig)
#         print(sig.size()) #1,10,1,1
        sig = torch.flatten(sig, 1)
#         print(sig.size())#1,10
        sig = self.linear1(sig)

        return sig

In [90]:
# Construct our model by instantiating the class defined above
x = make_example_data(14, size=4)
model = TwoLayerNet()

In [91]:
# for param in model.named_parameters():
#     print(param)

In [92]:
x = make_example_data(14, size=4)
y = torch.tensor([1], dtype=torch.float32)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(5):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)
    
    print("PRED: ", y_pred)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print("\n")

Parameter containing:
tensor([3.3548, 4.3685, 3.1820, 0.8569, 7.7401, 1.9282, 7.5094, 1.6299, 4.6097,
        7.3130], requires_grad=True)
PRED:  tensor([[103.8194]], grad_fn=<AddmmBackward>)


Parameter containing:
tensor([3.3548, 4.3685, 3.1820, 0.8569, 7.7401, 1.9282, 7.5094, 1.6299, 4.6097,
        7.3130], requires_grad=True)
PRED:  tensor([[225642.1875]], grad_fn=<AddmmBackward>)


Parameter containing:
tensor([3.3548, 4.3685, 3.1820, 0.8569, 7.7401, 1.9282, 7.5094, 1.6299, 4.6097,
        7.3130], requires_grad=True)
PRED:  tensor([[3.2169e+15]], grad_fn=<AddmmBackward>)


Parameter containing:
tensor([3.3548, 4.3685, 3.1820, 0.8569, 7.7401, 1.9282, 7.5094, 1.6299, 4.6097,
        7.3130], requires_grad=True)
PRED:  tensor([[nan]], grad_fn=<AddmmBackward>)


Parameter containing:
tensor([3.3548, 4.3685, 3.1820, 0.8569, 7.7401, 1.9282, 7.5094, 1.6299, 4.6097,
        7.3130], requires_grad=True)
PRED:  tensor([[nan]], grad_fn=<AddmmBackward>)




/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [82]:
hey = nn.Parameter(torch.Tensor(1, 1, 2))
b = nn.Parameter(torch.Tensor(1))

print(hey)
# print(b)

# init params
nn.init.xavier_uniform_(hey)
# b.data.uniform_(-1, 1)

Parameter containing:
tensor([[[7.0065e-45, 0.0000e+00]]], requires_grad=True)


Parameter containing:
tensor([[[-0.3727, -0.6900]]], requires_grad=True)

In [83]:
hey = torch.nn.Parameter(torch.tensor(random.choices(range(0,outDim[1]), k=16), dtype=torch.float32)) 

# nn.init.xavier_uniform_(hey)

torch.nn.init.calculate_gain(hey)

ValueError: Unsupported nonlinearity Parameter containing:
tensor([8., 5., 5., 6., 5., 2., 6., 5., 6., 1., 3., 4., 2., 6., 3., 8.],
       requires_grad=True)

In [87]:
w = torch.nn.Parameter(torch.empty(10))
nn.init.uniform_(w, a = 0, b = 8)


Parameter containing:
tensor([4.1101, 1.6600, 2.3353, 3.6779, 2.9621, 4.4281, 7.5158, 6.6865, 1.7108,
        1.7685], requires_grad=True)

In [31]:
for param in model.named_parameters():
    print(param)

('alpha', Parameter containing:
tensor(0.5000, requires_grad=True))
('coords', Parameter containing:
tensor([6., 0., 4., 3., 7., 7., 7., 8., 4., 7., 3., 1., 8., 0., 6., 0.],
       requires_grad=True))
('conv1.weight', Parameter containing:
tensor([[[[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]]],


        [[[-inf, -inf, -inf, -inf, 

In [665]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
x = SocialSig()

In [666]:
'''
Make a layer that has a x trainabale params as the coordinates
    -- THIS LAYER NEEDS TO HAVE A FORWARD PASS
    (https://discuss.pytorch.org/t/pytorch-equivalent-of-keras/29412)
'''
 

'\nMake a layer that has a x trainabale params as the coordinates\n    -- THIS LAYER NEEDS TO HAVE A FORWARD PASS\n    (https://discuss.pytorch.org/t/pytorch-equivalent-of-keras/29412)\n'